In [0]:
import pandas as pd
import sqlite3


In [43]:

con = sqlite3.connect('/content/imdb.db')
X=pd.read_sql_query("""select COUNT(*) from movie""", con)
print("Total number of movie reviews:",X['COUNT(*)'][0])



Total number of movie reviews: 3475


In [0]:

cur = con.cursor()
cur.execute("""Update movie SET year=Replace(Replace(Replace(Replace(Replace(Replace(year,'III',''),'II',''),'IV',''),'I',''),'XV',''),'V','')""")
cur.execute("""Update movie SET year=Replace(Replace(year,')',''),'(','')""")
cur.execute("""Update movie SET year=TRIM(year)""")
cur.execute("""Update m_producer SET pid=TRIM(pid)""")
cur.execute("""Update person SET name=TRIM(name)""")
cur.execute("""Update m_cast SET pid=TRIM(pid)""")
cur.execute("""Update m_director SET pid=TRIM(pid)""")
cur.close()



**QUESTION 1**

In [0]:
pd.read_sql_query("""

SELECT movie.title as title,movie.year as year,person.name as producer
FROM M_producer
INNER JOIN movie
        ON M_producer.MID= movie.MID
INNER JOIN Person 
        ON M_producer.PID= Person.PID
where movie.mid in 
(select mid from movie where (((year%4)=0 AND (year%100)!=0) OR (year%400)=0) and mid in 
(select mid from M_Genre where gid in 
(select genre_id from genre where genre LIKE '%Comedy%')))


""", con).head()

,title,year,producer
0,Mastizaade,2016,Rangita Pritish Nandy
1,Mastizaade,2016,Anoop Ram Kumar
2,Mastizaade,2016,Pritish Nandy
3,Harold & Kumar Go to White Castle,2004,J. Miles Dale
4,Harold & Kumar Go to White Castle,2004,Joseph Drake


**QUESTION 2**

In [0]:
pd.read_sql_query("""

select person.name from 
m_cast join person
on m_cast.pid=person.pid
join movie
on m_cast.mid=movie.mid
where movie.title='Anand'

""", con).head()

,name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Brahm Bhardwaj
3,Ramesh Deo
4,Seema Deo


**QUESTION 3**

In [48]:
pd.read_sql_query("""

SELECT distinct p1.name as actor
FROM M_cast
INNER JOIN movie
        ON M_cast.MID= movie.MID
INNER JOIN Person p1
        ON M_cast.PID= p1.PID
where p1.pid in (
  
SELECT person.pid 
FROM M_cast
INNER JOIN movie
        ON M_cast.MID= movie.MID
INNER JOIN Person 
        ON M_cast.PID= Person.PID
where movie.year <1970 

INTERSECT 

SELECT person.pid 
FROM M_cast
INNER JOIN movie
        ON M_cast.MID= movie.MID
INNER JOIN Person 
        ON M_cast.PID= Person.PID
where movie.year>1990      
)


""", con).head(100)

,actor
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni
5,Rakesh Sharma
6,Sanjay Dutt
7,Ric Young
8,Yusuf
9,Suhasini Mulay


**QUESTION 4**

In [30]:
pd.read_sql_query("""

SELECT person.name,count(distinct movie.mid) as total_films
FROM M_director
INNER JOIN movie
        ON M_director.MID= movie.MID
INNER JOIN Pe
        ON M_director.PID= Person.PID
group by m_director.pid
having total_films>=10
order by total_films
desc        
""", con).head(20)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



DatabaseError: ignored

**QUESTION 5 PART 1**

In [0]:
pd.read_sql_query("""

select movie.year as Year,count(*) as Total_movies_with_only_female
from movie
WHERE not exists
(
select * from
m_cast,person
where
m_cast.mid=movie.mid and m_cast.pid=person.pid and person.gender='Male'
)

group by movie.year
""", con)


,Year,Total_movies_with_only_female
0,2018,1
1,1939,1
2,1999,1
3,2000,1
4,2009,1
5,2012,1
6,2018,1


**QUESTION 5 PART 2**

In [0]:
pd.read_sql_query("""

select female_count.year as Year,((female_count.Total_movies_with_only_female)*100)/total_count.Total as Percentage
from
((select movie.year as Year,count(*) as Total_movies_with_only_female
from movie
WHERE not exists
(
select * from
m_cast,person
where
m_cast.mid=movie.mid and m_cast.pid=person.pid and person.gender='Male'
)
group by movie.year) female_count,
(select movie.year,count(*) as Total
 from movie
 group by movie.year) total_count)
where female_count.year=total_count.year

""", con).head(20)

,Year,Percentage
0,2018,9
1,1939,50
2,1999,1
3,2000,1
4,2009,1
5,2012,0
6,2018,1


**QUESTION 6**

In [0]:
pd.read_sql_query("""

SELECT movie.title,COUNT(distinct person.pid) as cast_size
FROM M_Cast
INNER JOIN movie
        ON M_Cast.MID= movie.MID
INNER JOIN Person 
        ON M_Cast.PID= Person.PID
Group by movie.mid
Order by cast_size DESC
""", con).head(5)

,title,cast_size
0,Ocean's Eight,238
1,Apaharan,233
2,Gold,215
3,My Name Is Khan,213
4,Captain America: Civil War,191


**QUESTION 7**

In [0]:
pd.read_sql_query("""

select
(SUBSTR(cast(movie.year as varchar(4)),1,3) || '0s') as Decade,
count(Distinct movie.mid) as Total_movies
from movie
group by SUBSTR(cast(year as varchar(4)),1,3)
order by Total_movies desc

""", con).head(5)

,Decade,Total_movies
0,2010s,1018
1,2000s,959
2,1990s,551
3,1980s,342
4,1970s,254


**QUESTION 8**

In [0]:
pd.read_sql_query("""

select Name from Person where not exists
(select distinct(PID) from M_Cast as C1 natural join Movie as M1 
where exists
(select MID from M_Cast as C2 natural join Movie as M2 where C1.PID = C2.PID and 
(M1.year - 3) > M2.year and not exists
(select MID from M_Cast as C3 natural join Movie as M3 
where C1.PID = C3.PID and M1.year>M3.year and M3.year>M2.year)))
""", con).head(100)

,name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis
5,Peter Mullan
6,Jack Reynor
7,Eddie Marsan
8,Tom Hollander
9,Matthew Rhys


**QUESTION 9**

In [0]:
pd.read_sql_query("""
select name from
(
select pa.name,count(d.mid) as yash_total from
person pd join
m_director d on
d.pid=pd.pid inner join
m_cast on d.mid=m_cast.mid
join person pa on
pa.pid=m_cast.pid
where pd.name='Yash Chopra'
group by pd.name,pa.name
having not exists(

select count(d_1.mid) as other_directors from
person pd_1 join
m_director d_1 on
d_1.pid=pd_1.pid inner join
m_cast on d_1.mid=m_cast.mid
join person pa_1 on
pa_1.pid=m_cast.pid
where pd_1.name!='Yash Chopra'
group by pd_1.name,pa_1.name
having pa.pid=pa_1.pid and yash_total<other_directors
)
)
""", con)
  

,name
0,Abbie Murphy
1,Achala Sachdev
2,Akhtar Mirza
3,Akhtar-Ul-Iman
4,Aloka Mukherjee
5,Amarinder Sodhi
6,Amir Zadey
7,Amrit
8,Andrew Bicknell
9,Anick Wiget


**QUESTION 10**

In [0]:
pd.read_sql_query("""
select distinct person.name from m_cast
join person on
m_cast.pid=person.pid
where m_cast.mid in
(select distinct m_cast.mid from m_cast
join person p1 on
m_cast.pid=p1.pid
where p1.pid in
(select distinct person.pid from m_cast
join person on
m_cast.pid=person.pid
where person.name!='Shah Rukh Khan' and m_cast.mid in
(select distinct mid from m_cast
join person on
m_cast.pid=person.pid
where person.name='Shah Rukh Khan')))

and person.pid not in

(select distinct person.pid from m_cast
join person on
m_cast.pid=person.pid
where person.pid in
(select distinct person.pid from m_cast
join person on
m_cast.pid=person.pid
where person.name!='Shah Rukh Khan' and m_cast.mid in
(select distinct mid from m_cast
join person on
m_cast.pid=person.pid
where person.name='Shah Rukh Khan')))

""", con)
  

,name
0,Alicia Vikander
1,Dominic West
2,Walton Goggins
3,Daniel Wu
4,Kristin Scott Thomas
5,Derek Jacobi
6,Alexandre Willaume
7,Tamer Burjaq
8,Adrian Collins
9,Keenan Arrison
